### Quantifying Uncertainty in Medical Diagnoses

#### Predicting Side Effects

In [23]:
import numpy as np
import pgmpy
from itertools import product
import pgmpy.inference
import pgmpy.factors
import pgmpy.models
import pgmpy.extern
import warnings
warnings.filterwarnings('ignore')

Step 1: Define network structure by adding all nodes to the Bayesian model.

In [24]:
# Starting with defining the network structure
from pgmpy.models import BayesianModel
from pgmpy.inference import BeliefPropagation

#add nodes
sideEffects_model = BayesianModel([('SideEffect', 'Rash'), 
                                  ('Allergies', 'Rash'),
                                  ('Rash', 'Xerostomia'),
                                  ('Rash', 'Hives'),
                                  ('Anxiety', 'Xerostomia')])
 

Step 2: Define conditional probability distributions.

In [25]:
from pgmpy.factors.discrete import TabularCPD

cpd_sideEffects = TabularCPD(variable='SideEffect', variable_card=2,
                          values=[[0.99],  #!side effects
                                  [0.01]]) #side effects

cpd_rash = TabularCPD(variable='Rash', variable_card=2,
                       values=[[0.999, 0.06, 0.71, 0.05],
                               [0.001, 0.94, 0.29, 0.95]], 
                       evidence=['Allergies', 'SideEffect'], 
                       evidence_card=[2, 2])

cpd_allergies = TabularCPD(variable='Allergies',
                            variable_card=2,
                            values=[[0.98],   #!allergies
                                    [0.02]])  #allergies

cpd_anxiety = TabularCPD(variable='Anxiety',
                          variable_card=2,
                          values=[[0.95],   #!anxiety
                                  [0.05]])  #anxiety

cpd_xerostomia = TabularCPD(variable='Xerostomia',
                           variable_card=2,
                           values=[[0.99, 0.50, 0.10, 0.05],#!dry mouth
                               [0.01, 0.50, 0.90, 0.95]],   #dry mouth
                           evidence=['Rash', 'Anxiety'],
                           evidence_card=[2, 2])

cpd_hives = TabularCPD(variable='Hives',
                           variable_card=2,
                           values=[[0.99, 0.30],  #!hives
                                   [0.01, 0.70]], #hives
                           evidence=['Rash'],
                           evidence_card=[2])

Step 3: Add conditional probabilities and check model

In [26]:
# Associate the CPDs with the network
sideEffects_model.add_cpds(cpd_sideEffects, cpd_rash, cpd_allergies, cpd_anxiety, cpd_xerostomia, cpd_hives)
# Check network structure and CPDs and verify that probabilities sum to 1
sideEffects_model.check_model()

True

In [27]:
# Example: Look at CPD of Alarm
print(sideEffects_model.get_cpds('Rash'))

+------------+--------------+--------------+--------------+--------------+
| Allergies  | Allergies_0  | Allergies_0  | Allergies_1  | Allergies_1  |
+------------+--------------+--------------+--------------+--------------+
| SideEffect | SideEffect_0 | SideEffect_1 | SideEffect_0 | SideEffect_1 |
+------------+--------------+--------------+--------------+--------------+
| Rash_0     | 0.999        | 0.06         | 0.71         | 0.05         |
+------------+--------------+--------------+--------------+--------------+
| Rash_1     | 0.001        | 0.94         | 0.29         | 0.95         |
+------------+--------------+--------------+--------------+--------------+


Step 4: Analyze network independencies

In [28]:
# Get all local dependencies in the network
print(sideEffects_model.local_independencies(['Rash','SideEffect','Rash','Xerostomia','Hives','Anxiety']))

(Rash _|_ Anxiety | SideEffect, Allergies)
(SideEffect _|_ Anxiety, Allergies)
(Rash _|_ Anxiety | SideEffect, Allergies)
(Xerostomia _|_ Hives, SideEffect, Allergies | Anxiety, Rash)
(Hives _|_ Anxiety, SideEffect, Xerostomia, Allergies | Rash)
(Anxiety _|_ Hives, SideEffect, Rash, Allergies)


Step 5: Active trails

In [29]:
 print(sideEffects_model.active_trail_nodes('Allergies'))

{'Allergies': {'Hives', 'Xerostomia', 'Rash', 'Allergies'}}


Step 6: Inference in Bayesian networks

Before posing new queries to the network, I will verify the existing queries

In [30]:
belief_propagation = BeliefPropagation(sideEffects_model)

In [31]:
q = belief_propagation.query(['Rash'],evidence={'Allergies':1}) 
print(q['Rash'])

+--------+-------------+
| Rash   |   phi(Rash) |
+========+=============+
| Rash_0 |      0.7034 |
+--------+-------------+
| Rash_1 |      0.2966 |
+--------+-------------+


In [32]:
q = belief_propagation.query(['Rash'], evidence={'Allergies':0}) 
print(q['Rash'])

+--------+-------------+
| Rash   |   phi(Rash) |
+========+=============+
| Rash_0 |      0.9896 |
+--------+-------------+
| Rash_1 |      0.0104 |
+--------+-------------+


Causal inference:

In [33]:
#Query: Given the observation of side effects, what is the probability of dry mouth?
q = belief_propagation.query(['Xerostomia'], evidence={'SideEffect': 1}) 
print(q['Xerostomia'])

+--------------+-------------------+
| Xerostomia   |   phi(Xerostomia) |
+==============+===================+
| Xerostomia_0 |            0.1494 |
+--------------+-------------------+
| Xerostomia_1 |            0.8506 |
+--------------+-------------------+


In [34]:
#Query: Given allergies, what is the probability of having Hives?
q = belief_propagation.query(['Hives'], evidence={'Allergies': 1}) 
print(q['Hives'])

+---------+--------------+
| Hives   |   phi(Hives) |
+=========+==============+
| Hives_0 |       0.7853 |
+---------+--------------+
| Hives_1 |       0.2147 |
+---------+--------------+


Diagnostic inference:

In [35]:
#Query: Given that a patient has both a rash and hives, what is the probability for a side effect occuring?
q = belief_propagation.query(['SideEffect'], evidence={'Rash': 1,'Hives': 1}) 
print(q['SideEffect'])

+--------------+-------------------+
| SideEffect   |   phi(SideEffect) |
+==============+===================+
| SideEffect_0 |            0.4165 |
+--------------+-------------------+
| SideEffect_1 |            0.5835 |
+--------------+-------------------+


Intercausal inference:

In [38]:
#Query: Given both a rash and allergies, what is the probability for a side effect?
q = belief_propagation.query(['SideEffect'], evidence={'Rash': 1,'Allergies': 1}) 
print(q['SideEffect'])

+--------------+-------------------+
| SideEffect   |   phi(SideEffect) |
+==============+===================+
| SideEffect_0 |            0.9680 |
+--------------+-------------------+
| SideEffect_1 |            0.0320 |
+--------------+-------------------+


In [39]:
#Query: Given a patient with dry mouth, what is the probability for anxiety being the cause?
q = belief_propagation.query(['Anxiety'], evidence={'Xerostomia': 1}) 
print(q['Anxiety'])

+-----------+----------------+
| Anxiety   |   phi(Anxiety) |
+===========+================+
| Anxiety_0 |         0.4769 |
+-----------+----------------+
| Anxiety_1 |         0.5231 |
+-----------+----------------+


Mixed inference:

In [40]:
#Query: Given a patient with dry mouth and no allergies, what is the probability of getting a rash? 
#Diagnostic and causal inference
q = belief_propagation.query(['Rash'], evidence={'Xerostomia': 1, 'Allergies': 0}) 
print(q['Rash'])

+--------+-------------+
| Rash   |   phi(Rash) |
+========+=============+
| Rash_0 |      0.7845 |
+--------+-------------+
| Rash_1 |      0.2155 |
+--------+-------------+


Step 7: Making Decisions

In [41]:
#Query 1: What is the cause for dry mouth if there is no side effect?
q = belief_propagation.map_query(variables=['SideEffect','Anxiety'], evidence={'Xerostomia': 1, 'Allergies': 0})
print(q)

{'SideEffect': 0, 'Anxiety': 1}


In [42]:
#Query 2: And what if the patient also has hives?
q = belief_propagation.map_query(variables=['SideEffect','Anxiety'], 
                                 evidence={'Xerostomia': 1,'Hives': 1,'Allergies': 0})
print(q)

{'SideEffect': 1, 'Anxiety': 0}
